# Smart case study

## import libraly

In [1]:
import gdown
import pandas as pd
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import inflection
from catboost import CatBoostRegressor, Pool
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import joblib


## Dowload dataset from google disk

In [ ]:
# URL to the file on Google Drive
url_rents = 'https://drive.google.com/uc?id=1oc_RJRsQEiJutVdWjlRdbJ773YpE9h6x'
output_rents = '../data/snp_dld_2024_rents.csv'
url_trans = 'https://drive.google.com/uc?id=1liNykIOnfR5KRR4MXJISCZCKJYFnXQC1'
output_trans = '../data/snp_dld_2024_transactions.csv'
# Download the file
gdown.download(url_rents, output_rents, quiet=False)
gdown.download(url_trans, output_trans, quiet=False)

# Load data into DataFrame
snp_dld_2024_rents = pd.read_csv(output_rents)
snp_dld_2024_transactions = pd.read_csv(output_trans)
print(snp_dld_2024_rents.shape)
print(snp_dld_2024_transactions.shape)

Downloading...
From (original): https://drive.google.com/uc?id=1oc_RJRsQEiJutVdWjlRdbJ773YpE9h6x
From (redirected): https://drive.google.com/uc?id=1oc_RJRsQEiJutVdWjlRdbJ773YpE9h6x&confirm=t&uuid=5828f42d-7e33-417d-adf2-327a1fb2a318
To: /media/aleksey/A6B828A60EB3956D/Github/SB-Case-Study-Materials/data/snp_dld_2024_rents.csv
100%|██████████| 278M/278M [01:05<00:00, 4.26MB/s] 


## Preprocess

### Rentals

In [ ]:
def load_data(path):
    """
    Загрузка данных из CSV файлов.
    
    :param rentals_path: Путь к файлу с данными по аренде.
    :param sales_path: Путь к файлу с данными по продажам.
    :return: Два DataFrame с данными по аренде и продажам.
    """
    df = pd.read_csv(path)

    return df

rentals = load_data(output_rents)

/tmp/ipykernel_678557/4169113296.py:9: DtypeWarning: Columns (35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


In [ ]:
def preprocess_data_rents(df):
    """
    Data preprocessing: handling missing data, encoding categorical variables, and scaling numerical features.
    
    :param df: DataFrame with raw data.
    :return: DataFrame with preprocessed data.
    """

    # Convert date columns to datetime
    df[['registration_date', 'contract_start_date', 'contract_end_date', 'req_from', 'req_to']] = df[['registration_date', 'contract_start_date', 'contract_end_date', 'req_from', 'req_to']].apply(pd.to_datetime)
    
    # Replace 't'/'f' with True/False
    df['is_freehold'] = df['is_freehold'].replace({'t': True, 'f': False})
    
    # Calculate time deltas
    df['delta_time_registration'] = (df['registration_date'] - df['contract_start_date']).dt.days
    df['delta_time_reg_from'] = (df['registration_date'] - df['req_from']).dt.days
    df['time_reg'] = (df['req_to'] - df['req_from']).dt.days
    df['time_contract'] = (df['contract_end_date'] - df['contract_start_date']).dt.days

    # Sort by version number and drop duplicates

    df.sort_values(['contract_amount', 'version_number','registration_date'], ascending=False, inplace=True)
    
    # Filter out rows with non-positive contract amounts
    df = df.query('contract_amount > 10').copy()

    df = df.drop_duplicates(subset=['ejari_contract_number', 'version_number', 'contract_start_date', 'contract_end_date'], keep='first')

    # Drop unnecessary columns
    df.drop(['is_freehold_text', 'land_property_id', 'property_id', 'property_type_ar', 
             'property_subtype_ar', 'ejari_contract_number',
             'property_usage_ar', 'property_usage_id', 'project_name_ar', 'area_ar', 'nearest_landmark_ar', 
             'nearest_metro_ar', 'nearest_mall_ar', 'master_project_ar', 'ejari_property_type_id', 
             'ejari_property_sub_type_id', 'meta_ts', 'area_id', 'version_text','annual_amount',
             'contract_end_date', 'contract_start_date', 'registration_date', 'req_from', 'req_to'
            ], axis=1, inplace=True)
    
    # Identify numerical and categorical features
    numerical_features = df.select_dtypes(include=['int64', 'float64']).columns

        # Replace NaN values in categorical features with 'missing'
    # Replace NaN values in categorical features with 'missing'
    df = df.apply(lambda x: x.fillna('missing') if x.dtype == 'object' else x)
        
    categorical_features = df.select_dtypes(include=['object']).columns

    return df, numerical_features, categorical_features



In [ ]:
df_preprocess_data_rents, numerical_features_rents, categorical_features_rents = preprocess_data_rents(rentals)
df_preprocess_data_rents

,version_number,contract_amount,is_freehold,property_size_sqm,parcel_id,property_type_en,property_subtype_en,property_usage_en,total_properties,rooms,...,area_en,nearest_landmark_en,nearest_metro_en,nearest_mall_en,master_project_en,entry_id,delta_time_registration,delta_time_reg_from,time_reg,time_contract
645663,1,4.410800e+08,True,47258.20,3920513.0,Building,Building,Residential,1,NaN,...,Marsa Dubai,Burj Al Arab,Jumeirah Beach Resdency,Marina Mall,missing,94375,1852,25,30,3287
55150,1,1.963500e+08,False,33600.00,1290104.0,Building,Building,Commercial,1,NaN,...,Port Saeed,Dubai International Airport,Deira City Centre,Dubai Mall,missing,94368,40,10,30,3651
475651,1,1.927818e+08,True,1854.36,NaN,Unit,Office,Commercial,4,NaN,...,Zaabeel First,Burj Khalifa,Al Jafiliya Metro Station,Dubai Mall,missing,94373,203,20,29,3651
475652,1,1.927818e+08,True,1854.29,NaN,Unit,Office,Commercial,4,NaN,...,Zaabeel First,Burj Khalifa,Al Jafiliya Metro Station,Dubai Mall,missing,94373,203,20,29,3651
475653,1,1.927818e+08,True,1854.57,NaN,Unit,Office,Commercial,4,NaN,...,Zaabeel First,Burj Khalifa,Al Jafiliya Metro Station,Dubai Mall,missing,94373,203,20,29,3651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
671589,12,1.000000e-02,True,118.20,2514313.0,Unit,Flat,Residential,1,NaN,...,Mirdif,Dubai International Airport,Rashidiya Metro Station,City Centre Mirdif,missing,94375,295,21,30,365
587543,10,1.000000e-02,True,66.44,2514114.0,Unit,Flat,Residential,1,NaN,...,Mirdif,Dubai International Airport,Rashidiya Metro Station,City Centre Mirdif,missing,94375,280,21,30,365
588961,7,1.000000e-02,True,71.67,2514114.0,Unit,Flat,Residential,1,NaN,...,Mirdif,Dubai International Airport,Rashidiya Metro Station,City Centre Mirdif,missing,94375,182,21,30,365
608641,3,1.000000e-02,True,73.32,NaN,Unit,Flat,Residential,1,NaN,...,Al Khairan First,Dubai International Airport,Creek Metro Station,City Centre Mirdif,missing,94375,-10,21,30,90


### Trans

In [ ]:
trans = load_data(output_trans)

In [ ]:
def preprocess_data_trans(df):
    """
    Data preprocessing: handling missing data, encoding categorical variables, and scaling numerical features.
    
    :param df: DataFrame with raw data.
    :return: DataFrame with preprocessed data.
    """

    df['is_offplan']= df['is_offplan'].replace({'t': True, 'f': False})
    df['is_freehold'] = df['is_freehold_text'].replace({'Free Hold': True, 'Non Free Hold': False})
    df.drop_duplicates(subset=['transaction_number', 'transaction_datetime', 'req_from', 'req_to', 'amount', 
                            'registration_type_en'], keep='last', inplace=True)

        # Filter out rows with non-positive contract amounts
    df = df.query('amount > 0').copy()
        # Identify numerical and categorical features
    # Calculate time deltas
    df[['req_to', 'transaction_datetime', 'req_from']] = df[[ 'transaction_datetime', 'req_from', 'req_to']].apply(pd.to_datetime)

    df['delta_time_reg_to'] = (df['req_to'] - df['transaction_datetime']).dt.days
    df['delta_time_reg_from'] = (df['req_from'] - df['transaction_datetime'] ).dt.days
    df['time_reg'] = (df['req_from'] - df['req_to']).dt.days

    # Sort by version number and drop duplicates
    df.sort_values(['transaction_datetime', 'amount'], ascending=False, inplace=True)
    df.drop_duplicates(subset=['transaction_number', 'transaction_datetime', 'req_from', 'req_to', 'amount', 
                                'registration_type_en'], keep='last', inplace=True)

    df.drop(['property_usage_id', 'transaction_subtype_id', 'transaction_type_id', 'property_id', 
            'transaction_subtype_en', 'is_freehold_text',
            'property_type_ar','property_type_id', 'property_subtype_ar',
            'property_subtype_id', 'building_age', 'rooms_ar', 'project_name_ar',
            'area_ar', 'area_id', 'nearest_landmark_ar', 'nearest_metro_ar', 'nearest_mall_ar',
            'master_project_ar', 'meta_ts', 'transaction_number',
            'req_from', 'transaction_datetime', 'req_to'
            ], axis=1, inplace=True)   

    numerical_features = df.select_dtypes(include=['int64', 'float64']).columns
    # Replace NaN values in categorical features with 'missing'
    df = df.apply(lambda x: x.fillna('missing') if x.dtype == 'object' else x)
        
    categorical_features = df.select_dtypes(include=['object']).columns
    return df, numerical_features, categorical_features

In [ ]:
df_preprocess_data_trans, numerical_features_trans, categorical_features_trans = preprocess_data_trans(trans)
df_preprocess_data_trans

,transaction_type_en,registration_type_en,property_usage_en,amount,total_buyer,total_seller,transaction_size_sqm,property_size_sqm,parcel_id,is_offplan,...,project_name_en,area_en,nearest_landmark_en,nearest_metro_en,nearest_mall_en,master_project_en,entry_id,delta_time_reg_to,delta_time_reg_from,time_reg
159419,Mortgage,Ready,Residential,2.228449e+09,0,0,7432243.24,7432243.24,NaN,False,...,missing,Mugatrah,missing,missing,missing,missing,94367,24,24,-1
159658,Mortgage,Ready,Commercial,5.520000e+08,0,0,10703.66,10703.66,NaN,False,...,missing,Al Thanyah Fifth,missing,missing,missing,missing,94367,23,24,0
162252,Mortgage,Ready,Residential,5.203507e+08,0,0,31384.41,31384.41,NaN,False,...,missing,DUBAI INTERNATIONAL ACADEMIC CITY,missing,missing,City Centre Mirdif,missing,94367,5,24,18
157251,Mortgage,Ready,Residential,5.030000e+08,0,0,12897.82,13747.52,6008142.0,False,...,missing,CITY OF ARABIA,IMG World Adventures,missing,missing,missing,94367,4,24,19
159615,Mortgage,Ready,Residential,4.000000e+08,0,0,5416.09,5416.09,3450566.0,False,...,missing,BUSINESS BAY,Downtown Dubai,Business Bay Metro Station,Dubai Mall,missing,94367,12,24,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8669,Sales,Ready,Residential,6.195700e+04,1,1,205.22,205.22,NaN,False,...,"THE ROYAL ATLANTIS,RESORT AND RESIDENCES",PALM JUMEIRAH,Burj Al Arab,Al Sufouh,Marina Mall,missing,94359,30,30,-1
14895,Gifts,Ready,Residential,5.995989e+04,0,0,1.00,1136.76,1260815.0,False,...,missing,Abu Hail,Dubai International Airport,Abu Baker Al Siddique Metro Station,missing,missing,94359,9,30,20
8888,Sales,Ready,Residential,5.804500e+04,1,1,8.55,225.66,1271225.0,False,...,missing,Hor Al Anz,Dubai International Airport,Abu Hail Metro Station,City Centre Mirdif,missing,94359,22,30,7
14684,Gifts,Ready,Residential,3.017709e+04,0,0,12.34,136.53,3260865.0,False,...,D1,JADDAF WATERFRONT,Dubai International Airport,Al Jadaf Metro Station,City Centre Mirdif,missing,94359,22,30,7


## Model creation

### catboost_model_rents

In [ ]:
# Assuming 'contract_amount' is the target variable
target_column_rents = 'contract_amount'

In [ ]:
def cross_validate_model(X, y, cat_features, cv=5):
    """
    Perform cross-validation on the CatBoost model and return the best model and evaluation metrics.
    
    :param X: DataFrame with features.
    :param y: Series with target variable.
    :param cat_features: List of categorical feature names.
    :param cv: Number of cross-validation folds.
    :return: Dictionary with RMSE, R2, MAE scores, best model.
    """
    model = CatBoostRegressor(loss_function='RMSE', verbose=0)
    param_grid = {
        'iterations': [100, 200],
        'depth': [4, 6, 8],
        'learning_rate': [0.01, 0.1]
    }
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=cv, scoring='neg_mean_squared_error', verbose=1)
    grid_search.fit(X, y, cat_features=cat_features)
    
    best_model = grid_search.best_estimator_
    
    # Cross-validation predictions
    y_pred = cross_val_score(best_model, X, y, cv=cv, scoring='neg_mean_squared_error')
    
    # Calculate metrics
    rmse_scores = np.sqrt(-y_pred)
    r2_scores = cross_val_score(best_model, X, y, cv=cv, scoring='r2')
    mae_scores = cross_val_score(best_model, X, y, cv=cv, scoring='neg_mean_absolute_error')
    
    metrics = {
        'RMSE': (np.mean(rmse_scores), np.std(rmse_scores)),
        'R2': (np.mean(r2_scores), np.std(r2_scores)),
        'MAE': (np.mean(-mae_scores), np.std(-mae_scores))
    }
    
    return metrics, best_model

In [ ]:
def train_and_evaluate_model(df, target_column, categorical_features):
    """
    Train and evaluate a CatBoost model.
    
    :param df: DataFrame with preprocessed data.
    :param target_column: Name of the target column.
    :param categorical_features: List of categorical feature names.
    :return: Trained model.
    """
    # Remove rows with NaN values in the target column
    df = df.dropna(subset=[target_column])
    
    X = df.drop(columns=[target_column])
    y = df[target_column]
    
    # Perform cross-validation and get the best model
    metrics, best_model = cross_validate_model(X, y, categorical_features)
    print(f"Cross-validated RMSE: {rmse:.2f}")
    
    # Train the best model on the entire dataset
    best_model.fit(X, y, cat_features=categorical_features)
    
    return best_model, metrics


In [ ]:
def save_model(model, filename):
    model.save_model(f'../models/{filename}', format="cbm")

In [ ]:

# Train and evaluate model
model_rents, metrics_rents = train_and_evaluate_model(df_preprocess_data_rents, target_column_rents, categorical_features_rents.tolist())

# Save the model
save_model(model_rents, 'catboost_model_rents.cbm')

Fitting 5 folds for each of 12 candidates, totalling 60 fits


ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "_catboost.pyx", line 2286, in _catboost.get_float_feature
  File "_catboost.pyx", line 1139, in _catboost._FloatOrNan
  File "_catboost.pyx", line 951, in _catboost._FloatOrNanFromString
TypeError: Cannot convert 'b'Unit'' to float

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/aleksey/anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/aleksey/anaconda3/lib/python3.10/site-packages/catboost/core.py", line 5730, in fit
    return self._fit(X, y,

In [ ]:
model_rents

In [ ]:
metrics

### catboost_model_trans

In [ ]:
# Assuming 'contract_amount' is the target variable
target_column_trans = 'amount'

# Train and evaluate model
model_trans, metrics_trans = train_and_evaluate_model(df_preprocess_data_trans, target_column_trans, categorical_features_trans.tolist())

# Save the model
save_model(model_trans, 'catboost_model_trans.cbm')

In [ ]:
model_rents

In [ ]:
metrics